<a href="https://colab.research.google.com/github/StayFrostea/LearningML/blob/main/Roszell_Thesis_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the first baseline model of the MRI Thesis

### Importing the dataset from Googler Drive

In [ ]:
## Loading the google drive where I stored the MOSMEDDATA files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
toTrain_path = '/content/drive/MyDrive/Colab Notebooks/Data/'
toPredict_path = '/content/drive/MyDrive/Colab Notebooks/Data/'

toTrain_path_output = '/content/drive/MyDrive/Colab Notebooks/Data/'
toPredict_path_output = '/content/drive/MyDrive/Colab Notebooks/Data/'

### Sectioning off some of the data for prediction

In [ ]:
## A tool for spliting the image files before processing
!pip install split-folders
import splitfolders

## 80/20 Split of the data
splitfolders.ratio(toTrain_path, output=toTrain_path_output, seed=1337, ratio=(0.8, 0.2))
splitfolders.ratio(toPredict_path, output=toPredict_path_output, seed=1337, ratio=(0.8, 0.2))

In [ ]:
import os

## First class files
path, dirs, files = next(os.walk(abnormal_path_output + '/train/class1'))
file_count = len(files)
file_count

In [ ]:
## Second class files
path, dirs, files = next(os.walk(abnormal_path_output + '/train/class2'))
file_count = len(files)
file_count

In [ ]:
## Read Function
def read_NifTi(fp):
    scan = nib.load(fp)
    scan = scan.get_fdata()
    return scan

## Resize function
def resizeVolume(img):

    ## desired
    d_depth = 64
    d_width = 150
    d_height = 150

    ## current
    c_depth = img.shape[-1]
    c_width = img.shape[0]
    c_height = img.shape[1]

    ## factor to change by
    d_factor = d_depth/c_depth
    w_factor = d_width/c_width
    h_factor = d_height/c_height

    ## Adjust proper rotation
    img = ndimage.rotate(img, 90, reshape = False)

    ## apply the factors
    img = ndimage.zoom(img, (w_factor, h_factor, d_factor), order = 1)

    return img
  
## Normalize Function
def normalizeVolume(vol):
    min = -1000
    max = 400
    vol[vol < min] = min
    vol[vol > max] = max
    vol = (vol - min) / (max - min)
    vol = vol.astype("float32")
    return vol

## Processing Function
def processVolume(path):
    volume = read_NifTi(path)
    volume = normalizeVolume(volume)
    volume = resizeVolume(volume)
    return volume

In [ ]:
## Setting up the filepaths for each file in class 1
normal_scan_paths = [
    os.path.join(os.getcwd(), normal_path_output + '/train/class1', x)
    for x in os.listdir(normal_path_output + '/train/class1')
]

## Setting up the filepaths for each file in class 2
alzheimer_scan_paths = [
    os.path.join(os.getcwd(), normal_path_output + '/train/class2', x)
    for x in os.listdir(normal_path_output + '/train/class2')
]

In [ ]:
## Normal Subject files into numpy arrays
normal_volumes = np.array([processVolume(path) for path in normal_scan_paths])
normal_volume_labels = np.array([0 for _ in range(len(normal_volumes))])

In [ ]:
## Alzheimer's Subject files into numpy arrays
alzheimer_volumes = np.array([processVolume(path) for path in alzheimer_scan_paths])
alzheimer_volume_labels = np.array([1 for _ in range(len(alzheimer_volumes))])

In [ ]:
print("CT scans with normal lung tissue: " + str(len(normal_scan_paths)))
print("CT scans with abnormal lung tissue: " + str(len(alzheimer_scan_paths)))

In [ ]:
## 60/20 Split of volumes
X_train = np.concatenate((abnormal_volumes[:60], normal_volumes[:60]), axis=0)
y_train = np.concatenate((abnormal_volume_labels[:60], normal_volume_labels[:60]), axis=0)

X_val = np.concatenate((abnormal_volumes[60:], normal_volumes[60:]), axis=0)
y_val = np.concatenate((abnormal_volume_labels[60:], normal_volume_labels[60:]), axis=0)

print(   
"Number of samples in train and validation are %d and %d."
    % (X_train.shape[0], X_val.shape[0])
)
X_train.shape

### Preprocessing Directives

In [ ]:
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
## Rotation Augmentation
def rotate(volume):

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume

In [ ]:
## For now we will not do preprocessing
## Only add the the stack to match dimensionality

def train_preprocess(volume, label):
  # volume = rotate(volume)
  volume = tf.stack((volume,)*3, axis = 3)
  return volume, label
def valid_preprocess(volume, label):
  volume = tf.stack((volume,)*3, axis = 3)
  return volume, label

In [ ]:
## Run the preprocessing Steps
X_train_r, y_train_r = train_preprocess(X_train, y_train)
X_val_r, y_val_r = valid_preprocess(X_val, y_train)

In [ ]:
## Slicing Function
def threeDToTwoD(threeDVol, numVol):

  twoDVol = np.zeros((numVol*64, 150, 150, 3), np.float32)

  count = 0

  for i in range(numVol):
    for j in range(64):
      twoDVol[count] = threeDVol[i,:,:,:,j]
      count = count + 1

  return twoDVol

In [ ]:
## Extending Labels to Match
def extendLabels(labelArr, numVol):

  newLabelArr = np.zeros((numVol * 64), np.float32)

  for i in range(numVol):
    for j in range(64):
      newLabelArr[(i*64)+j] = labelArr[i]

  return newLabelArr

In [ ]:
## New arrays

## First the training images
X_train_f = threeDToTwoD(X_train_r, 120)

## Second the validation images
X_val_f = threeDToTwoD(X_val_r, 40)

## Third the training labels
y_train_f = extendLabels(y_train_r, 120)

## Fourth the validation labels
y_val_f = extendLabels(y_val_r, 40)

### Now the model

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
data_augmentation = tf.keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),])

In [ ]:
## The model build
def buildModel():

  initial_model = tf.keras.applications.Xception(
      weights = 'imagenet',
      input_shape = (150,150,3),
      include_top = False)
  
  ## Freeze the pretrained model parameters
  initial_model.trainable = False

  inputs = tf.keras.Input(shape = (150,150,3))

  x = data_augmentation(inputs)

  scale_layer = tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
  x = scale_layer(x)

  x = initial_model(inputs, training = False)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(inputs, outputs)
  return model

model = buildModel()
model.summary()

In [ ]:
## Setting up the fit parameters
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics = tf.keras.metrics.BinaryAccuracy(),
              )

## Defining checkpoints
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "3D_CT_classification.h5", save_best_only=True
)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

## How man runs
epochs = 20

In [ ]:
## Training the model
model.fit(x = X_train_f,
          y = y_train_f,
          validation_data=(X_val_f, y_val_f),
          epochs=epochs,
          shuffle=True,
          verbose='auto',
          callbacks = [ checkpoint_cb , early_stopping_cb],
          )